# The General Claims Data
- Primary question of interest: Who was at fault?
<br> <br>
- Secondary questions of interest:
    - What Claim Group should the claim be routed to?
    - Can we predict severity type (minor, moderate, major, life threatening, death)
    - What was the cause of the loss?

In [1]:
import pandas as pd
import numpy as np
import spacy
import en_core_web_lg

In [2]:
import nltk
from nltk import word_tokenize

In [3]:
from nltk.corpus import stopwords
# stop = stopwords.words('english')

In [4]:
stop = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 
 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 
 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 
 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 
 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 
 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 
 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'doing', 'it', 'how', 'further', 'was', 
 'here', 'than']

In [5]:
import gensim

In [65]:
# general_tokens_df = pd.read_csv('data/general_with_tokens.csv')

In [66]:
# general_tokens_df.head()

In [67]:
from ast import literal_eval

In [68]:
# def join_strings(df, columns):
#     n_df = df
#     for col in columns:
#         def join_col(row):
#             claim_list = row[col]
#             return ' '.join(claim_list)
#         n_df[col] = n_df.apply(join_col, axis = 1)
#     return n_df
# convert lists from strings back to lists
def convert_to_list(df,cols):
    n_df = df
    for col in cols:
        def de_list(row):
            list_str = row[col]
            return literal_eval(list_str)
        n_df[col] = n_df.apply(de_list, axis = 1)
    return n_df

In [69]:
# general_tokens_df = convert_to_list(general_tokens_df, ['claim_level_list', 'exposure_level_list',
#                                                        'accident_desc_list', 'injury_desc_list',
#                                                        'damage_desc_list'])

In [8]:
general_df = pd.read_csv('data/NSS_DS_data.thegeneral.csv')


In [9]:
general_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida


In [10]:
general_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120657 entries, 0 to 120656
Data columns (total 14 columns):
ClaimID_RGEN           120657 non-null int64
ExposureID_RGEN        120657 non-null int64
ClaimLevelBody         120657 non-null object
ExposureLevelBody      111199 non-null object
AccidentDescription    120657 non-null object
InjuryDescription      120619 non-null object
DamageDescription      120657 non-null object
SeverityTypeName       87699 non-null object
ClaimGroup             120642 non-null object
FaultRatingName        120656 non-null object
LossCauseName          120657 non-null object
CCCreateTime           120657 non-null object
CloseDate              99611 non-null object
StateName              120657 non-null object
dtypes: int64(2), object(12)
memory usage: 12.9+ MB


In [11]:
general_no_unknown = general_df.loc[general_df.loc[:,'FaultRatingName'] != 'Fault unknown',:].reset_index(drop=True)

In [12]:
general_no_unknown = general_no_unknown.fillna({'ExposureLevelBody':' ', 'InjuryDescription':' '})

In [13]:
general_no_unknown.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119272 entries, 0 to 119271
Data columns (total 14 columns):
ClaimID_RGEN           119272 non-null int64
ExposureID_RGEN        119272 non-null int64
ClaimLevelBody         119272 non-null object
ExposureLevelBody      119272 non-null object
AccidentDescription    119272 non-null object
InjuryDescription      119272 non-null object
DamageDescription      119272 non-null object
SeverityTypeName       86721 non-null object
ClaimGroup             119257 non-null object
FaultRatingName        119271 non-null object
LossCauseName          119272 non-null object
CCCreateTime           119272 non-null object
CloseDate              99601 non-null object
StateName              119272 non-null object
dtypes: int64(2), object(12)
memory usage: 12.7+ MB


In [14]:
nlp = en_core_web_lg.load()

In [15]:
def column_to_list_text(df, col_name):
    return df[col_name].tolist()

In [16]:
def lower_columns(df, col_list):
    n_df = df
    for col_name in col_list:
        n_col = col_name + '_preprocess'
        n_df[n_col] = n_df[col_name].str.lower()
    return n_df

In [17]:
lowered_df = lower_columns(general_no_unknown, ['ClaimLevelBody', 'ExposureLevelBody', 'AccidentDescription', 
                                        'InjuryDescription', 'DamageDescription'])

In [18]:
lowered_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none


In [19]:
def tokenize_claim_level(row):
#     list_cols = [['ClaimLevelBody_preprocess', 'ExposureLevelBody_preprocess', 'AccidentDescription_preprocess', 
#                                         'InjuryDescription_preprocess', 'DamageDescription_preprocess']]
    text = row['ClaimLevelBody_preprocess']
    text_tokenize = [token for token in word_tokenize(text) if token.isalpha() and token not in stop]
    return text_tokenize
        
def pre_process(df, col_list):
    for col in col_list:
        df[col] = df.apply(lambda x: tokenize(x, col), axis=1)
    return df

In [20]:
#lowered_df['ClaimLevelBody_preprocess'] = lowered_df.apply(tokenize_claim_level, axis = 1)

In [21]:
#out_df.head()

In [118]:
lowered_df.ClaimLevelBody[0]

'reviewed\r\ncoverage  sc rated policy and loss effective dates of coverage  ******* ********\r\nni was operating a vnop  vehicle appears to be a loaner veh policy was set to cx for non payment but reinstated on ******* without a lapse send ror citing the definition of your covered auto and insuring agreement under section * liability coverage \r\n\r\nliability modified compneg state need statements and police report to complete the lia investigation based on the report of the loss other driver pulled into nis row \r\n\r\nexposure  other driver appears to have allstate lets confirm and rule out umpdumbi  the exposure appears to be pd rule out injuries to clmnt driver determine the severity of nis injuries secure uim rejection form\r\n\r\naction plan \r\nsend ror\r\ncomplete vnop investigation statements confirm ownership of veh and insurance infomation on the vnop\r\nconfirm other driver has lia coverage rule out umpdumbi\r\ncomplete liability investigation statementspolice reportset a

In [22]:
lowered_df.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,LossCauseName,CCCreateTime,CloseDate,StateName,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,Collision with motor vehicle,2015-03-12 09:05:17.9100000,2015-04-29 11:25:18.3190000,South Carolina,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,Collision while merging / lane change,2015-03-12 11:46:23.1590000,2015-07-06 08:26:47.5350000,Florida,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,Rear-end collision,2015-03-12 13:12:35.4440000,2017-11-20 14:17:39.0910000,Florida,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,Collision with motor vehicle,2015-03-12 16:35:17.3570000,2016-01-29 13:54:25.6150000,Florida,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none


In [70]:
fault_ratings = column_to_list_text(lowered_df, 'FaultRatingName')

In [71]:
claim_level_body = column_to_list_text(lowered_df, 'ClaimLevelBody_preprocess')

In [72]:
exposure_level_body = column_to_list_text(lowered_df, 'ExposureLevelBody_preprocess')

In [73]:
accident_description = column_to_list_text(lowered_df, 'AccidentDescription_preprocess')

In [74]:
injury_description = column_to_list_text(lowered_df, 'InjuryDescription_preprocess')

In [75]:
damage_description = column_to_list_text(lowered_df, 'DamageDescription_preprocess')

In [76]:
claim_level_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in claim_level_body]
exposure_level_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in exposure_level_body]
accident_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in accident_description]
injury_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                           for doc in injury_description]
damage_desc_tokenized = [[token for token in word_tokenize(doc) if token.isalpha() and token not in stop] 
                         for doc in damage_description]

# [token for token in word_tokenize(damage_description) if token.isalpha()]

In [77]:
from collections import Counter

In [114]:
claim_level_tokenized[0]

['reviewed',
 'coverage',
 'sc',
 'rated',
 'policy',
 'loss',
 'effective',
 'dates',
 'coverage',
 'ni',
 'operating',
 'vnop',
 'vehicle',
 'appears',
 'loaner',
 'veh',
 'policy',
 'set',
 'cx',
 'non',
 'payment',
 'reinstated',
 'without',
 'lapse',
 'send',
 'ror',
 'citing',
 'definition',
 'covered',
 'auto',
 'insuring',
 'agreement',
 'section',
 'liability',
 'coverage',
 'liability',
 'modified',
 'compneg',
 'state',
 'need',
 'statements',
 'police',
 'report',
 'complete',
 'lia',
 'investigation',
 'based',
 'report',
 'loss',
 'driver',
 'pulled',
 'nis',
 'row',
 'exposure',
 'driver',
 'appears',
 'allstate',
 'lets',
 'confirm',
 'rule',
 'umpdumbi',
 'exposure',
 'appears',
 'pd',
 'rule',
 'injuries',
 'clmnt',
 'driver',
 'determine',
 'severity',
 'nis',
 'injuries',
 'secure',
 'uim',
 'rejection',
 'form',
 'action',
 'plan',
 'send',
 'ror',
 'complete',
 'vnop',
 'investigation',
 'statements',
 'confirm',
 'ownership',
 'veh',
 'insurance',
 'infomation',


In [80]:
word_counts_claim = [dict(Counter(doc)) for doc in claim_level_tokenized]
word_counts_exposure = [dict(Counter(doc)) for doc in exposure_level_tokenized]
word_counts_accident = [dict(Counter(doc)) for doc in accident_desc_tokenized]
word_counts_injury = [dict(Counter(doc)) for doc in injury_desc_tokenized]
word_counts_damage = [dict(Counter(doc)) for doc in damage_desc_tokenized]

In [133]:
word_counts_claim[3]

{'insured': 4,
 'hospital': 7,
 'lost': 2,
 'service': 2,
 'middle': 2,
 'claim': 30,
 'c': 4,
 'dol': 26,
 'vehiclelisted': 2,
 'yes': 8,
 'driverlisted': 2,
 'valid': 4,
 'dl': 4,
 'policy': 13,
 'state': 4,
 'limits': 2,
 'period': 2,
 'fl': 8,
 'pip': 19,
 'w': 2,
 'ded': 9,
 'cc': 40,
 'inception': 6,
 'loss': 4,
 'prior': 4,
 'losses': 2,
 'none': 6,
 'within': 2,
 'days': 2,
 'exclusions': 2,
 'listed': 2,
 'verified': 4,
 'coverages': 2,
 'insd': 4,
 'pending': 9,
 'l': 4,
 'documented': 2,
 'support': 2,
 'decision': 2,
 'cv': 18,
 'af': 6,
 'careless': 2,
 'driving': 2,
 'negligence': 2,
 'doctrine': 2,
 'pure': 2,
 'comp': 2,
 'facts': 2,
 'rear': 4,
 'ended': 4,
 'iv': 32,
 'occupants': 4,
 'id': 14,
 'gp': 2,
 'cd': 18,
 'e': 4,
 'reserves': 4,
 'collision': 2,
 'action': 4,
 'plan': 4,
 'rs': 26,
 'igp': 2,
 'pr': 22,
 'verify': 23,
 'su': 16,
 'estimate': 16,
 'liability': 30,
 'r': 4,
 'siu': 4,
 'referral': 4,
 'subro': 2,
 'salvage': 2,
 'na': 4,
 'storage': 2,
 'obc'

In [128]:
iv_list = [0] * len(word_counts_claim)
liability_list = [0] * len(word_counts_claim)
dcv_list = [0] * len(word_counts_claim)
inj_list = [0] * len(word_counts_claim)
vnop_list = [0] * len(word_counts_claim)
ac_list = [0] * len(word_counts_claim)
cv_list = [0] * len(word_counts_claim)
div_list = [0] * len(word_counts_claim)
insd_list = [0] * len(word_counts_claim)
insured_list = [0] * len(word_counts_claim)
clmt_list = [0] * len(word_counts_claim)

In [129]:
for ind, count_dict in enumerate(word_counts_claim):
    if 'iv' in count_dict:
        iv_list[ind] = count_dict['iv']
    else:
        iv_list[ind] = 0
    if 'liability' in count_dict:
        liability_list[ind] = count_dict['liability']
    else:
        liability_list[ind] = 0
    if 'dcv' in count_dict:
        dcv_list[ind] = count_dict['dcv']
    else:
        dcv_list[ind] = 0
    if 'inj' in count_dict:
        inj_list[ind] = count_dict['inj']
    else:
        inj_list[ind] = 0
    if 'vnop' in count_dict:
        vnop_list[ind] = count_dict['vnop']
    else:
        vnop_list[ind] = 0
    if 'ac' in count_dict:
        ac_list[ind] = count_dict['ac']
    else:
        ac_list[ind] = 0
    if 'cv' in count_dict:
        cv_list[ind] = count_dict['cv']
    else:
        cv_list[ind] = 0
    if 'div' in count_dict:
        div_list[ind] = count_dict['div']
    else:
        div_list[ind] = 0
    if 'insd' in count_dict:
        insd_list[ind] = count_dict['insd']
    else:
        insd_list[ind] = 0
    if 'insured' in count_dict:
        insured_list[ind] = count_dict['insured']
    else:
        insured_list[ind] = 0

In [117]:
iv_list[:10]

[42, 8, 30, 30, 14, 14, 16, 7, 2, 2]

In [110]:
# word_counts_claim[2]

In [130]:
word_cols = {'iv_count': iv_list,
         'liability_count': liability_list,
         'dcv_count': dcv_list,
         'inj_count': inj_list,
             'vnop_count': vnop_list,
            'ac_count': ac_list,
            'cv_count': cv_list,
            'div_count': div_list,
            'insd_count': insd_list,
            'insured_count': insured_list}
df_spec_word_count = pd.DataFrame.from_dict(word_cols)

In [131]:
df_spec_word_count.head()

,iv_count,liability_count,dcv_count,inj_count,vnop_count,ac_count,cv_count,div_count,insd_count,insured_count
0,38,42,52,14,42,20,32,34,0,0
1,6,8,0,0,0,6,14,0,0,2
2,32,30,0,0,0,0,18,0,4,4
3,32,30,0,0,0,0,18,0,4,4
4,3,14,6,6,3,28,12,0,0,10


In [108]:
dict_cols = {'word_counts_claim': word_counts_claim,
         'word_counts_exposure': word_counts_exposure,
         'word_counts_accident': word_counts_accident,
         'word_counts_injury': word_counts_injury,
             'word_counts_damage': word_counts_damage}
df_word_count = pd.DataFrame.from_dict(dict_cols)

In [109]:
df_word_count.head()

,word_counts_claim,word_counts_exposure,word_counts_accident,word_counts_injury,word_counts_damage
0,"{'reviewed': 6, 'coverage': 18, 'sc': 22, 'rat...","{'none': 2, 'liability': 2, 'accepted': 2, 'by...","{'insured': 1, 'test': 1, 'driving': 1, 'vehic...",{'neckback': 1},"{'front': 1, 'left': 2, 'side': 2, 'damages': ..."
1,"{'c': 4, 'dol': 4, 'tol': 6, 'pm': 2, 'venue':...","{'c': 2, 'coverage': 8, 'dol': 4, 'fnol': 2, '...","{'iv': 2, 'driving': 1, 'road': 1, 'ov': 1, 't...",{'pain': 1},"{'front': 2, 'right': 1, 'headlight': 1, 'side..."
2,"{'insured': 4, 'hospital': 7, 'lost': 2, 'serv...","{'none': 2, 'c': 2, 'coverage': 3, 'dol': 2, '...","{'insured': 1, 'stopped': 1, 'stop': 1, 'light...",{'strain': 1},"{'unknown': 1, 'damages': 1, 'rear': 2, 'bumpe..."
3,"{'insured': 4, 'hospital': 7, 'lost': 2, 'serv...","{'none': 2, 'hhcsi': 2, 'dos': 2, 'atd': 2, 'b...","{'insured': 1, 'stopped': 1, 'stop': 1, 'light...",{'back': 1},"{'unknown': 1, 'damages': 1, 'rear': 2, 'bumpe..."
4,"{'adjuster': 10, 'office': 23, 'reassigned': 2...","{'new': 2, 'pip': 19, 'loss': 4, 'received': 2...","{'ov': 1, 're': 1, 'host': 1, 'veh': 1}","{'neck': 1, 'back': 1}",{'none': 3}


In [84]:
token_cols = {'claim_level_list': claim_level_tokenized,
         'exposure_level_list': exposure_level_tokenized,
         'accident_desc_list': accident_desc_tokenized,
         'injury_desc_list': injury_desc_tokenized,
             'damage_desc_list': damage_desc_tokenized}
df = pd.DataFrame.from_dict(token_cols)

In [85]:
# df.head()

In [86]:
# len(df)

In [134]:
df_spec_word_counts = pd.concat([df_with_counts, df_spec_word_count], axis = 1)

In [87]:
df_with_tokens = pd.concat([lowered_df, df], axis = 1)

In [111]:
df_with_counts = pd.concat([general_full_strings, df_word_count], axis = 1)

In [ ]:
df_spec_word_counts.head()

In [ ]:
# len(df_with_tokens)

In [113]:
general_full_strings.to_csv('data/general_with_counts.csv', index = False)

In [90]:
# lowered_df.FaultRatingName.value_counts()


In [91]:
# general_tokens_df.iloc[0,22]

In [92]:
def join_strings(df, columns):
    n_df = df
    for col in columns:
        def join_col(row):
            claim_list = row[col]
            return ' '.join(claim_list)
        n_df[col] = n_df.apply(join_col, axis = 1)
    return n_df

In [93]:
general_full_strings = join_strings(df_with_tokens, ['claim_level_list', 'exposure_level_list',
                                                       'accident_desc_list', 'injury_desc_list',
                                                       'damage_desc_list'])

In [94]:
general_full_strings.head()

,ClaimID_RGEN,ExposureID_RGEN,ClaimLevelBody,ExposureLevelBody,AccidentDescription,InjuryDescription,DamageDescription,SeverityTypeName,ClaimGroup,FaultRatingName,...,ClaimLevelBody_preprocess,ExposureLevelBody_preprocess,AccidentDescription_preprocess,InjuryDescription_preprocess,DamageDescription_preprocess,claim_level_list,exposure_level_list,accident_desc_list,injury_desc_list,damage_desc_list
0,10000,131005,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,Moderate,Tampa Field Ops,Other party at fault,...,reviewed\r\ncoverage sc rated policy and loss...,none none liability accepted by adverse and da...,the insured was test driving a vehicle the ov...,neckback,front left side damages towed collision tow i...,reviewed coverage sc rated policy loss effecti...,none none liability accepted by adverse damage...,insured test driving vehicle ov pulled parking...,neckback,front left side damages towed collision tow gr...
1,10001,131006,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,Minor,Tampa Field Ops,Other party at fault,...,c dol *******\r\n tol *** pm\r\n v...,c coverage\r\ndol *****\r\nfnol ***** by ni \...,the iv was driving down the road when the ov i...,pain,front right headlight front side of bumper dr...,c dol tol pm venue fl location jacksonville fl...,c coverage dol fnol by ni policy state fl loss...,iv driving road ov turning lane suddenly swerv...,pain,front right headlight front side bumper drivab...
2,10002,131007,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,Minor,Tampa Field Ops,Other party at fault,...,insured was in the hospital and lost service ...,none none c coverage\r\ndol *****\r\nfnol ***...,insured was stopped at the stop light when cv ...,strain,unknown damages rear bumper trunk right rear ...,insured hospital lost service middle claim ins...,none none c coverage dol fnol by ni policy sta...,insured stopped stop light cv rear ended iv,strain,unknown damages rear bumper trunk right rear l...
3,10003,131008,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,NaN,Tampa Field Ops,Other party at fault,...,insured was in the hospital and lost service ...,none none hhcsi dos ****** atd for brant willi...,insured was stopped at the stop light when cv ...,back,unknown damages rear bumper trunk right rear ...,insured hospital lost service middle claim ins...,none none hhcsi dos atd brant williams hhcsi d...,insured stopped stop light cv rear ended iv,back,unknown damages rear bumper trunk right rear l...
4,10004,131009,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,Minor,Fast Track,Other party at fault,...,adjuster out of office\r\nreassigned adjuster ...,new pip loss received additional action neede...,ov re host veh,neck back,none none none,adjuster office reassigned adjuster office rea...,new pip loss received additional action needed...,ov re host veh,neck back,none none none


In [119]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [120]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [121]:
X_train, X_test, y_train, y_test = train_test_split(accident_description, fault_ratings, 
                                                    test_size=0.25, random_state=42)

In [122]:
model = text_clf.fit(X_train, y_train)

In [123]:
predicted_MNB = text_clf.predict(X_test)

In [124]:
predicted_MNB[:5]

array(['Insured at fault', 'Insured at fault', 'Insured at fault',
       'Insured at fault', 'Insured at fault'], dtype='<U22')

In [125]:
np.mean(predicted_MNB == y_test)

0.6397142665504058

In [126]:
len(predicted_MNB)

29818

In [127]:
text_clf.score(X_test, y_test)

0.6397142665504058

In [128]:
from collections import Counter

In [129]:
Counter(predicted_MNB)

Counter({'Insured at fault': 28402,
         'Other party at fault': 912,
         'No fault': 404,
         'Comparative Negligence': 100})

In [98]:
28402/29818

0.9525119055603998

In [99]:
74304/len(general_no_unknown)

0.622979408411027